In [ ]:
####

In [ ]:
from langchain_groq import ChatGroq

import os
from dotenv import load_dotenv
load_dotenv()
llm_obj = ChatGroq(model="llama-3.1-8b-instant",api_key=os.getenv("GROQ_API_KEY"))

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter # RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

### Step-1
loader = TextLoader('my_docs.txt')
documents = loader.load()

### Step-2 
text_splitter = CharacterTextSplitter(chunk_size=500,chunk_overlap=50)
docs = text_splitter.split_documents(documents)

### Step-3
embeddings = OllamaEmbeddings(model='gemma:2b')


### Step-4
db = FAISS.from_documents(docs,embeddings)


## Step-5 (do query)
r = db.similarity_search('what is langchain?')
print(len(r))

In [ ]:
### Step-1
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA

# Step 1: Load documents
loader = TextLoader("my_docs.txt")
documents = loader.load()

# Step 2: Split text into chunks
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# Step 3: Create embeddings
embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Step 4: Store vectors
db = FAISS.from_documents(docs, embeddings)

#  Convert to retriever
retriever = db.as_retriever(search_kwargs={"k": 3})

# Step 5: Initialize LLM
llm_obj = Ollama(model='gemma:2b')

# Step 6: Create RetrievalQA chain
chain_obj = RetrievalQA.from_chain_type(
    llm=llm_obj,
    retriever=retriever,
    return_source_documents=True
)

# Step 7: Query
response1 = chain_obj("langchain?")
print(response1['result'])

response2 = chain_obj("how to write factorial program?")
print(response2['result'])